In [3]:
import nltk
import sklearn

In [65]:
import unicodedata
import re
stemmer = nltk.stem.RSLPStemmer()
stop = nltk.corpus.stopwords.words('portuguese')

acceptable = re.compile('[^a-z ]')
spaces = re.compile('[ ]+')

def preprocessor(txt):
    txt = txt.lower()
    txt = unicodedata.normalize('NFD', txt)
    txt = acceptable.sub("", txt)
    return txt

stop = {preprocessor(stemmer.stem(word)) for word in stop}

def analyzer(txt):
    txt = preprocessor(txt)
    tokens = spaces.split(txt)
    return [ stemmer.stem(token) for token in  tokens if len(token) >= 2 and token not in stop]


In [98]:
vetorizador = sklearn.feature_extraction.text.TfidfVectorizer(analyzer=analyzer, max_features=None, norm='l2')

In [96]:
import os
import  collections

textos_discursos = list()
discursos_metadado = dict()

diretorio = 'politicos/'

for file_name in os.listdir(diretorio):
    aux = file_name[:-4].split('_')
    estado = aux[0]
    partido = aux[1]
    nome = " ".join(aux[2:])
    
    if estado is None or partido is None:
        continue
        
    discursos = None
    with open(diretorio + file_name) as arq:
        linhas = arq.readlines()
        if len(linhas)//2 > 10:
            discursos = processa_discursos(linhas)
    
    if discursos is None:
        continue
        
    datas = list()
    for data, discurso in discursos:
        id_discurso = len(textos_discursos)
        textos_discursos.append(discurso)
        datas += [(data, id_discurso)]
        
    discursos_metadado[nome] = { 'partido': partido, 'estado': estado, 'datas': datas}

In [89]:
def filtro_fala(fala):
    if fala is not None and len(fala) > 10:
        return True
    return False

In [ ]:
def processa_discursos(linhas):
    i = 0
    discursos = list()
    data_corrente = None
    for linha in linhas:
        if i % 2 == 0:
            data_corrente = linha.strip('\n').strip(':')
        else:
            if filtro_fala(linha):
                discursos += [(data_corrente, linha)]
        i += 1
    return discursos


In [99]:
vetor_discursos = vetorizador.fit_transform(textos_discursos)

In [213]:
import numpy as np
from scipy.spatial.distance import cosine
import datetime
def analise_coerencia(nome_deputado, janela = 5):
    dados_deputado = discursos_metadado[nome_deputado]
    ids = list()
    datas = list()
    similaridades = list()
    
    sorted(dados_deputado['datas'], key= lambda elem: datetime.datetime.strptime(elem[0], "%d/%m/%Y %H:%M:%S") )
    
    for data, id_discurso in dados_deputado['datas']:
        ids += [id_discurso]
        datas += [data]
        
    vetores_analise = vetor_discursos[ids, :]
    for i in range(janela, len(ids)):
        contexto = vetores_analise[i-janela:i, :].mean(axis=0)
        atual = vetores_analise[i, :].toarray()
        
        similaridade = cosine(contexto, atual)
        similaridades += [similaridade]
        
    return datas[janela:], similaridades


In [214]:
def coerencia_highcharts(deputado, janela=30):
    datas, similaridades = analise_coerencia(deputado, janela)
    modelo = '''$(function () {
        $('#container').highcharts({
            title: {
                text: 'Coerência do Deputado (Janela = %s discursos)',
                x: -20 //center
            },
            xAxis: {
                categories: [%s]
            },
            yAxis: {
                title: {
                    text: 'Similaridade do Cosseno'
                },
                plotLines: [{
                    value: 0,
                    width: 1,
                    color: '#808080'
                }]
            },
            legend: {
                layout: 'vertical',
                align: 'right',
                verticalAlign: 'middle',
                borderWidth: 0
            },
            series: [{
                name: '%s',
                data: [%s]
            }]
        });
    });'''
    resultado = modelo % (janela, ', '.join(["'"+ d + "'" for d in datas]), deputado, ', '.join([ str(s) for s in similaridades]))
    return resultado

In [224]:
lista = [ datetime.datetime.strptime(i[0], "%d/%m/%Y %H:%M:%S") for i in discursos_metadado['CARLOS MANATO']['datas']]

In [228]:
print(coerencia_highcharts('JANDIRA FEGHALI'))

$(function () {
        $('#container').highcharts({
            title: {
                text: 'Coerência do Deputado (Janela = 30 discursos)',
                x: -20 //center
            },
            xAxis: {
                categories: ['23/9/2015 16:58:00', '23/9/2015 16:58:00', '22/9/2015 17:36:00', '22/9/2015 17:36:00', '22/9/2015 17:36:00', '16/9/2015 16:54:00', '10/9/2015 12:00:00', '10/9/2015 12:00:00', '10/9/2015 12:00:00', '9/9/2015 17:58:00', '9/9/2015 17:58:00', '9/9/2015 17:58:00', '9/9/2015 17:58:00', '9/9/2015 17:58:00', '9/9/2015 16:10:00', '2/9/2015 20:02:00', '26/8/2015 20:14:00', '26/8/2015 20:14:00', '26/8/2015 20:14:00', '26/8/2015 17:22:00', '26/8/2015 17:22:00', '26/8/2015 17:22:00', '26/8/2015 17:22:00', '19/8/2015 19:24:00', '19/8/2015 19:24:00', '18/8/2015 20:00:00', '18/8/2015 20:00:00', '11/8/2015 18:50:00', '11/8/2015 17:46:00', '11/8/2015 17:46:00', '6/8/2015 01:18:00', '16/7/2015 09:30:00', '14/7/2015 21:04:00', '14/7/2015 21:04:00', '14/7/2015 21:04:0